# Random Forest Classification

## Importing the libraries

In [1]:
import random
import numpy as np
import pandas as pd
import seaborn as sn
from numpy import std
from numpy import mean
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import confusion_matrix, accuracy_score
%matplotlib inline

In [2]:
%cd Z:\PPMI_Data\Excels\Only_Male\GeneCpG\ScopaSex
df = pd.read_csv('SexMaleCateg_Methylome_APPRDX.csv')
print(df.shape)
df.head(3)

Z:\PPMI_Data\Excels\Only_Male\GeneCpG\ScopaSex
(308, 10540)


,Sentrix,PATNO,HYS,Gender,APPRDX,SCOPA_Sex,cg17370322,cg15396799,cg15388598,cg14189621,...,cg00600684,cg13207036,cg05750276,cg02312170,cg17391830,cg13151449,cg13596132,cg26538046,cg18607338,cg14149699
0,200991620021_R01C01,3001,2,1,1,2,0.013535,0.242275,0.012080,0.024553,...,0.927369,0.030600,0.052903,0.053346,0.041092,0.902988,0.911403,0.033966,0.827098,0.030215
1,200973410167_R04C01,3007,2,1,1,0,0.012805,0.109881,0.018853,0.052400,...,0.920645,0.023915,0.023711,0.035630,0.027145,0.896837,0.889727,0.038348,0.836846,0.083898
2,200973410167_R05C01,3010,2,1,1,0,0.012898,0.113455,0.014117,0.046599,...,0.917370,0.020184,0.031970,0.039161,0.027965,0.923971,0.877362,0.037637,0.808158,0.042991


In [3]:
df1 = df.drop(['PATNO', 'Sentrix', 'HYS', 'Gender', 'SCOPA_Sex'], axis=1)
df1.head(2)

,APPRDX,cg17370322,cg15396799,cg15388598,cg14189621,cg15158045,cg09770095,cg05412028,cg26685941,cg25868675,...,cg00600684,cg13207036,cg05750276,cg02312170,cg17391830,cg13151449,cg13596132,cg26538046,cg18607338,cg14149699
0,1,0.013535,0.242275,0.012080,0.024553,0.929080,0.953919,0.091946,0.082563,0.015361,...,0.927369,0.030600,0.052903,0.053346,0.041092,0.902988,0.911403,0.033966,0.827098,0.030215
1,1,0.012805,0.109881,0.018853,0.052400,0.945216,0.927368,0.117943,0.098877,0.019414,...,0.920645,0.023915,0.023711,0.035630,0.027145,0.896837,0.889727,0.038348,0.836846,0.083898


In [4]:
df1 = df1.transpose()
df1.head(2)

,0,1,2,3,4,5,6,7,8,9,...,298,299,300,301,302,303,304,305,306,307
APPRDX,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,2.000000,2.000000,2.00000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000
cg17370322,0.013535,0.012805,0.012898,0.009889,0.011545,0.011926,0.010231,0.008801,0.013504,0.009881,...,0.010807,0.010139,0.01166,0.011381,0.010628,0.010432,0.012813,0.011248,0.011074,0.013962


In [5]:
df1.rename(columns=df1.iloc[0], inplace = True)


In [6]:
df1.index.name ='CpG'
df1.reset_index(inplace=True)
df1.head(2)

,CpG,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
0,APPRDX,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,2.000000,2.000000,2.00000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000
1,cg17370322,0.013535,0.012805,0.012898,0.009889,0.011545,0.011926,0.010231,0.008801,0.013504,...,0.010807,0.010139,0.01166,0.011381,0.010628,0.010432,0.012813,0.011248,0.011074,0.013962


In [7]:
df1 = df1.drop ([0], axis = 0)
df1.head(2)

,CpG,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
1,cg17370322,0.013535,0.012805,0.012898,0.009889,0.011545,0.011926,0.010231,0.008801,0.013504,...,0.010807,0.010139,0.011660,0.011381,0.010628,0.010432,0.012813,0.011248,0.011074,0.013962
2,cg15396799,0.242275,0.109881,0.113455,0.251859,0.149287,0.176164,0.190884,0.180457,0.194320,...,0.245186,0.253274,0.213514,0.054089,0.246093,0.230421,0.242080,0.143204,0.096550,0.183015


In [8]:
df2 = pd.read_csv ("SexMale_BorutaMale_Top50.csv")
df2.head(2)

,CpG
0,cg19380722
1,cg05817202


In [9]:
df3 = pd.merge (df1, df2, on= "CpG", how= "inner")
df3.head(2)

,CpG,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
0,cg22870386,0.828698,0.888004,0.869679,0.890114,0.875837,0.885505,0.890914,0.866537,0.849121,...,0.856590,0.877590,0.880750,0.873875,0.858635,0.858282,0.881535,0.852012,0.876373,0.877364
1,cg22807314,0.920441,0.926729,0.932652,0.934655,0.943429,0.941686,0.942670,0.931270,0.930600,...,0.934809,0.943984,0.933317,0.926732,0.937179,0.931800,0.933254,0.946326,0.919645,0.937398


In [10]:
df3 = df3.transpose()
df3.head(2)

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
CpG,cg22870386,cg22807314,cg11865360,cg05817202,cg24583287,cg00417288,cg26243905,cg21369812,cg17422176,cg13165076,...,cg23093116,cg15559456,cg14595995,cg07897734,cg02081976,cg24272338,cg00875272,cg02153334,cg13518031,cg03303515
1.0,0.828698,0.920441,0.279839,0.823899,0.919071,0.892366,0.837745,0.586499,0.117596,0.585931,...,0.384906,0.691127,0.888622,0.842938,0.068129,0.912405,0.026473,0.942443,0.901294,0.245769


In [11]:
df3.rename(columns=df3.iloc[0], inplace = True)


In [12]:
df3.index.name ='APPRDX'
df3.reset_index(inplace=True)
df3.head(2)

,APPRDX,cg22870386,cg22807314,cg11865360,cg05817202,cg24583287,cg00417288,cg26243905,cg21369812,cg17422176,...,cg23093116,cg15559456,cg14595995,cg07897734,cg02081976,cg24272338,cg00875272,cg02153334,cg13518031,cg03303515
0,CpG,cg22870386,cg22807314,cg11865360,cg05817202,cg24583287,cg00417288,cg26243905,cg21369812,cg17422176,...,cg23093116,cg15559456,cg14595995,cg07897734,cg02081976,cg24272338,cg00875272,cg02153334,cg13518031,cg03303515
1,1.0,0.828698,0.920441,0.279839,0.823899,0.919071,0.892366,0.837745,0.586499,0.117596,...,0.384906,0.691127,0.888622,0.842938,0.068129,0.912405,0.026473,0.942443,0.901294,0.245769


In [13]:
df3 = df3.drop ([0], axis = 0)
df3.head(2)

,APPRDX,cg22870386,cg22807314,cg11865360,cg05817202,cg24583287,cg00417288,cg26243905,cg21369812,cg17422176,...,cg23093116,cg15559456,cg14595995,cg07897734,cg02081976,cg24272338,cg00875272,cg02153334,cg13518031,cg03303515
1,1.0,0.828698,0.920441,0.279839,0.823899,0.919071,0.892366,0.837745,0.586499,0.117596,...,0.384906,0.691127,0.888622,0.842938,0.068129,0.912405,0.026473,0.942443,0.901294,0.245769
2,1.0,0.888004,0.926729,0.137535,0.849287,0.921326,0.93447,0.900153,0.625846,0.147173,...,0.384591,0.592664,0.91541,0.923446,0.057739,0.891393,0.026996,0.898963,0.895743,0.211177


In [14]:
df3.reset_index(inplace = True, drop = True)
df3.head(3)

,APPRDX,cg22870386,cg22807314,cg11865360,cg05817202,cg24583287,cg00417288,cg26243905,cg21369812,cg17422176,...,cg23093116,cg15559456,cg14595995,cg07897734,cg02081976,cg24272338,cg00875272,cg02153334,cg13518031,cg03303515
0,1.0,0.828698,0.920441,0.279839,0.823899,0.919071,0.892366,0.837745,0.586499,0.117596,...,0.384906,0.691127,0.888622,0.842938,0.068129,0.912405,0.026473,0.942443,0.901294,0.245769
1,1.0,0.888004,0.926729,0.137535,0.849287,0.921326,0.93447,0.900153,0.625846,0.147173,...,0.384591,0.592664,0.91541,0.923446,0.057739,0.891393,0.026996,0.898963,0.895743,0.211177
2,1.0,0.869679,0.932652,0.129196,0.877793,0.92007,0.932953,0.895253,0.57045,0.151643,...,0.492602,0.591926,0.92151,0.922936,0.068335,0.863804,0.024627,0.90742,0.890185,0.185063


In [15]:
df3.shape

(308, 51)

In [16]:
df3 ['APPRDX'] = df3 ['APPRDX'].apply(int)

## RanFor 1000 times, CpG only 

In [17]:
df3.head(2)

,APPRDX,cg22870386,cg22807314,cg11865360,cg05817202,cg24583287,cg00417288,cg26243905,cg21369812,cg17422176,...,cg23093116,cg15559456,cg14595995,cg07897734,cg02081976,cg24272338,cg00875272,cg02153334,cg13518031,cg03303515
0,1,0.828698,0.920441,0.279839,0.823899,0.919071,0.892366,0.837745,0.586499,0.117596,...,0.384906,0.691127,0.888622,0.842938,0.068129,0.912405,0.026473,0.942443,0.901294,0.245769
1,1,0.888004,0.926729,0.137535,0.849287,0.921326,0.93447,0.900153,0.625846,0.147173,...,0.384591,0.592664,0.91541,0.923446,0.057739,0.891393,0.026996,0.898963,0.895743,0.211177


In [18]:
df3.shape

(308, 51)

In [19]:
np.random.seed (1)
dframe1 = pd.DataFrame(columns=['Accuracy', 'Std_Dev'])
for i in range (1,1001) :
        print(i)
        dataset = df3.groupby('APPRDX').apply(lambda x: x.sample(90))
        X = dataset.iloc[:, 1:].values
        y = dataset.iloc[:, 0].values
        classifier = RandomForestClassifier(n_estimators = 100, criterion = 'entropy', random_state = 1)
        cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=5, random_state=1)
        n_scores = cross_val_score(classifier, X, y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
        dframe1 = dframe1.append({'Accuracy': mean(n_scores), 'Std_Dev': std(n_scores)}, ignore_index=True)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [20]:
dframe1.head(2)

,Accuracy,Std_Dev
0,0.675556,0.113507
1,0.674444,0.099387


In [21]:
dframe1.to_csv ("Sex_BorutaCpG_Male_1000RF.csv")